In [ ]:
import numpy as np
import os

pcd_dir = '/home/yishu/Azure_Kinect_ROS_Driver/src/pc_data_for_yishu'
pcd_paths = [os.path.join(pcd_dir, pcd_name) for pcd_name in os.listdir(pcd_dir)]

id = 2
# path = pcd_paths[id]
path = '/home/yishu/Azure_Kinect_ROS_Driver/src/pc_data_for_yishu/incorrect_toilet.npy'
# path = '/home/yishu/Azure_Kinect_ROS_Driver/src/pc_data_for_yishu/fridge_L_open_15.npy'
print(path)
pcd = np.load(path)

In [ ]:
import numpy as np
rot = np.array([[0, -1, 0], [1,  0, 0], [0,  0, 1]])
mean_x = pcd[:, 0].mean()
mean_y = pcd[:, 1].mean()
rot_pcd = pcd.copy()
rot_pcd[:, 0] -= mean_x
rot_pcd[:, 1] -= mean_y
# rot_pcd[:, 2] += 1
rot_pcd = rot_pcd@rot.T

In [ ]:
import numpy as np
idx_of_max_flow = 115
contact_point = np.array([-0.04013849, -1.1314487, 0.0044618])
contact_point_world = pcd[idx_of_max_flow]
flow_vector = np.array([0.0443, -0.4799, 0.8762])

In [ ]:
import torch
from scipy.spatial.transform import Rotation 

def get_goal_point_and_orientation(contact_point, flow_vector):
    goal_point = contact_point + 0.2 * flow_vector
    e_z_init = torch.tensor([0, 0, 1.0]).float().cuda()
    e_y = -flow_vector
    e_x = torch.linalg.cross(e_y, e_z_init)
    e_x = e_x / e_x.norm(dim=-1)
    e_z = torch.linalg.cross(e_x, e_y)
    R_goal = torch.stack([e_x, e_y, e_z], dim=1).cuda()
    R_gripper = torch.as_tensor(
        [
            [1, 0, 0],
            [0, 0, 1.0],
            [0, -1.0, 0],
        ]
    ).cuda()

    goal_orientation = Rotation.from_matrix((R_goal @ R_gripper).cpu()).as_quat()
    return goal_point, goal_orientation


def transform_flow_contact_point_goal_point_and_orientation_to_world(contact_point, goal_point, goal_orientation, mean_x, mean_y, flow_vector):
    # Formatting goal_point and goal_orientation to be in the same frame as the point cloud so that it can be visualized
    R = torch.tensor([[0, 1, 0], [-1, 0, 0], [0, 0, 1]]).float().cuda()

    # Add the mean in x and y
    goal_point[0] += mean_x
    goal_point[1] += mean_y
    contact_point[0] += mean_x
    contact_point[1] += mean_y

    goal_point = goal_point @ R.T
    contact_point = contact_point @ R.T
    
    goal_point = goal_point.cpu().numpy()
    goal_point = np.reshape(goal_point, (1, 3))
    contact_point = contact_point.cpu().numpy()
    contact_point = np.reshape(contact_point, (1, 3))

    goal_orientation = Rotation.from_quat(goal_orientation)
    goal_orientation = torch.from_numpy(goal_orientation.as_matrix()).float().cuda()
    goal_orientation = goal_orientation @ R.T
    goal_orientation = Rotation.from_matrix(goal_orientation.cpu()).as_quat()

    flow_vector = flow_vector @ R.T

    return contact_point, goal_point, goal_orientation, flow_vector

In [ ]:
contact_point = torch.from_numpy(contact_point).float().cuda()
flow_vector_normalized = torch.from_numpy(flow_vector).float().cuda()
goal_point, goal_orientation = get_goal_point_and_orientation(contact_point, flow_vector_normalized)
contact_point, goal_point, goal_orientation,flow_vector = transform_flow_contact_point_goal_point_and_orientation_to_world(contact_point, goal_point, goal_orientation, mean_x, mean_y, flow_vector_normalized)

In [ ]:
e_z_init = torch.tensor([0, 0, 1.0]).float().cuda()
e_y = -torch.tensor([0, 0, 1]).float().cuda()
e_x = torch.linalg.cross(e_y, e_z_init)
e_x = e_x / (e_x.norm(dim=-1)+1e-6)
e_z = torch.linalg.cross(e_x, e_y)
R_goal = torch.stack([e_x, e_y, e_z], dim=1).cuda()
goal_orientation = Rotation.from_matrix((R_goal).cpu()).as_quat()
print(R_goal)
print(goal_orientation)

### Visuals - model input space

In [ ]:
import open3d as o3d

In [ ]:
import plotly.graph_objects as go

# Define vectors
vectors = [
    dict(x=[contact_point[0], ], y=[contact_point[1], 2], z=[contact_point[2], 1], name='Vector 1'),
    dict(x=[0, 1], y=[0, 2], z=[0, 3], name='Vector 2'),
    dict(x=[0, 1], y=[0, 0], z=[0, 1], name='Vector 3')
]

# Create the figure
fig = go.Figure()

# Add vectors to plot
for v in vectors:
    fig.add_trace(go.Scatter3d(x=v['x'], y=v['y'], z=v['z'], mode='lines+markers+text', name=v['name']))

# Update layout for a nice aspect ratio
fig.update_layout(scene=dict(
    xaxis=dict(nticks=4, range=[-5,5]),
    yaxis=dict(nticks=4, range=[-5,5]),
    zaxis=dict(nticks=4, range=[-5,5])
), width=700)

# Show the plot
fig.show()
